# Blob ナレッジソースの作成

BLOB ナレッジ ソースは、Azure AI Search エージェントパイプラインでマルチモーダル Azure BLOB コンテンツのインデックス作成とクエリを実行するために必要なすべての情報を指定します。 これは個別に作成され、ナレッジエージェントによって参照され、エージェントまたはチャットボットが `retrieve` アクションを呼び出すときにクエリ時に使用されます。

既存のインデックスと修飾されたインデックスを指定する 検索インデックスナレッジソースとは対照的に、BLOB ナレッジソースでは、外部データ ソース (BLOB コンテナー) と、エンリッチメント パイプライン全体の作成に使用されるモデルとプロパティが指定されます。

- 生成されたデータ ソースは、BLOB コンテナーを指定します
- 生成されたスキルセット チャンクとマルチモーダル コンテンツのベクトル化
- 生成されたインデックスは、インデックス付きコンテンツを格納し、Agentic retrieval の条件を満たします
- 生成されたインデクサーによって、インデックス作成とエンリッチメント パイプラインが駆動されます

生成されたインデックスは、ナレッジエージェントによって使用されるコンテンツを提供します。


https://learn.microsoft.com/ja-jp/azure/search/search-knowledge-source-how-to-blob

In [ ]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    AzureBlobKnowledgeSource,
    AzureBlobKnowledgeSourceParameters,
    AzureOpenAIVectorizerParameters,
    AzureOpenAIVectorizer,
    IndexingSchedule,
    SearchResourceEncryptionKey,
    KnowledgeAgentAzureOpenAIModel
)
import datetime
import os
from dotenv import load_dotenv

load_dotenv()

In [ ]:
# 環境変数から設定を取得
AZURE_OPENAI_ENDPOINT= os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY= os.getenv("AZURE_OPENAI_API_KEY")

SEARCH_ENDPOINT = os.getenv("AZURE_SEARCH_ENDPOINT")
SEARCH_KEY = os.getenv("AZURE_SEARCH_KEY")
# OPENAI_ENDPOINT = os.getenv("OPENAI_ENDPOINT")
# OPENAI_KEY = os.getenv("OPENAI_KEY")

# Azure Blob Storage の接続文字列
CONNECTION_STRING = "DefaultEndpointsProtocol="

In [ ]:
search_api_version = "2025-08-01-preview"  # 11.7.0b1で使用される新しいAPIバージョン

# SearchIndexClientの初期化
search_client = SearchIndexClient(
    endpoint=SEARCH_ENDPOINT,
    credential=AzureKeyCredential(SEARCH_KEY),
    api_version=search_api_version
)

In [ ]:
# Azure OpenAI Embedding Modelの設定
embedding_model = AzureOpenAIVectorizer(
    vectorizer_name="text-embedding-vectorizer",
    parameters=AzureOpenAIVectorizerParameters(
        resource_url=AZURE_OPENAI_ENDPOINT,
        deployment_name="text-embedding-3-small",
        api_key=AZURE_OPENAI_API_KEY,
        model_name="text-embedding-3-small",
        auth_identity=None
    )
)

# Chat Completion Modelの設定（修正版）
chat_completion_model = KnowledgeAgentAzureOpenAIModel(
    azure_open_ai_parameters=AzureOpenAIVectorizerParameters(
        resource_url=AZURE_OPENAI_ENDPOINT, 
        deployment_name="gpt-4.1-mini", 
        api_key=AZURE_OPENAI_API_KEY,
        model_name="gpt-4.1-mini", 
        auth_identity=None
    )
)

# Ingestion Scheduleの設定
ingestion_schedule = IndexingSchedule(
    interval="P1D",  # 1日間隔
    start_time=datetime.datetime(2025, 9, 8, 0, 0, 0, tzinfo=datetime.timezone.utc)
)


In [ ]:

# Azure Blob Knowledge Source Parametersの設定
azure_blob_parameters = AzureBlobKnowledgeSourceParameters(
    connection_string=CONNECTION_STRING,
    container_name="docs1",
    folder_path=None,
    disable_image_verbalization=None,
    identity=None,
    embedding_model=embedding_model,
    chat_completion_model=chat_completion_model,
    ingestion_schedule=ingestion_schedule
)

# Azure Blob Knowledge Sourceの作成
knowledge_source = AzureBlobKnowledgeSource(
    name="your-blob-ks",
    description="このナレッジソースは、・・・のPDFページを含むBlobストレージコンテナから取得されます。",
    azure_blob_parameters=azure_blob_parameters,
    e_tag=None,
    encryption_key=None
)


In [ ]:

try:
    # Knowledge Sourceの作成（PUT操作）
    created_knowledge_source = search_client.create_or_update_knowledge_source(knowledge_source)
    print(f"Knowledge Source '{created_knowledge_source.name}' が正常に作成/更新されました")
    print(f"Kind: {created_knowledge_source.kind}")
    print(f"Description: {created_knowledge_source.description}")
    print(f"Container: {created_knowledge_source.azure_blob_parameters.container_name}")
    
    if created_knowledge_source.azure_blob_parameters.ingestion_schedule:
        print(f"Ingestion Schedule: {created_knowledge_source.azure_blob_parameters.ingestion_schedule.interval}")
        print(f"Start Time: {created_knowledge_source.azure_blob_parameters.ingestion_schedule.start_time}")
    
except Exception as e:
    print(f"Knowledge Source の作成中にエラーが発生しました: {e}")
    print(f"エラーの種類: {type(e).__name__}")